In [233]:
import pandas as pd
import numpy as np
import pylab
import seaborn as sns
import scipy.stats as st
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from sklearn.preprocessing import scale

startup=pd.read_csv("C:\\Users\\kp\\Pictures\\Assignments\\Multi Linear Regression\\50_Startups.csv",header=0)

# # Scatter plot between the variables along with histograms
# sns.pairplot(startup)
# plt.show()
startup.head(10)

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94
5,131876.90,99814.71,362861.36,New York,156991.12
6,134615.46,147198.87,127716.82,California,156122.51
7,130298.13,145530.06,323876.68,Florida,155752.60
8,120542.52,148718.95,311613.29,New York,152211.77
9,123334.88,108679.17,304981.62,California,149759.96


In [234]:
startup=pd.get_dummies(startup,columns=['State'])

#Dropping records which are too sensitive for the regressor to learn the pattern(Outliers) by Influencer Plot
startup.drop(startup.index[[49,19,45]],axis=0,inplace=True)

features=['R&D Spend', 'Marketing Spend']
target=['Profit']
startup.head(5)

,R&D Spend,Administration,Marketing Spend,Profit,State_California,State_Florida,State_New York
0,165349.20,136897.80,471784.10,192261.83,0,0,1
1,162597.70,151377.59,443898.53,191792.06,1,0,0
2,153441.51,101145.55,407934.54,191050.39,0,1,0
3,144372.41,118671.85,383199.62,182901.99,0,0,1
4,142107.34,91391.77,366168.42,166187.94,0,1,0


In [235]:
modified_startup=startup
modified_startup['Marketing Spend']=startup['Marketing Spend'].transform(func='sqrt')
modified_startup['R&D Spend']=startup['R&D Spend'].transform(func=scale)
# modified_startup['Administration']=startup['Administration'].transform(func=scale)
modified_startup.head(5)

,R&D Spend,Administration,Marketing Spend,Profit,State_California,State_Florida,State_New York
0,2.006919,136897.80,686.865416,192261.83,0,0,1
1,1.944721,151377.59,666.257105,191792.06,1,0,0
2,1.737745,101145.55,638.697534,191050.39,0,1,0
3,1.532738,118671.85,619.031195,182901.99,0,0,1
4,1.481536,91391.77,605.118517,166187.94,0,1,0


In [236]:
#Model Building

regressor=smf.ols("Profit~Q('R&D Spend')+Q('Marketing Spend')",data=modified_startup).fit()

#Summary
regressor.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.966
Model:                            OLS   Adj. R-squared:                  0.964
Method:                 Least Squares   F-statistic:                     621.7
Date:                Thu, 30 Apr 2020   Prob (F-statistic):           5.52e-33
Time:                        08:46:10   Log-Likelihood:                -482.82
No. Observations:                  47   AIC:                             971.6
Df Residuals:                      44   BIC:                             977.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept             1.012e+05   4471.624     22.643      0.000    9.22e+04     1.1e+05
Q('R&D Spend')         3.39e+04   1464.160     23.151      0.000    3.09e+04    3.68e+04
Q('Marketing Spend')    30.3729      9.699      3.131      0.003      10.825      49.920
==============================================================================
Omnibus:                        0.166   Durbin-Watson:                   1.526
Prob(Omnibus):                  0.920   Jarque-Bera (JB):                0.172
Skew:                          -0.122   Prob(JB):                        0.918
Kurtosis:                       2.832   Cond. No.                     2.05e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.05e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [237]:
preds=regressor.predict(modified_startup[features])
# sm.graphics.influence_plot(regressor,figsize=(15,10))
# plt.show()

In [238]:
# #Normality Test for Residuals
# #In Order to have a Model's good fit it is important to have the Residuals follow a Normal Distribution Pattern

# # Normal Distribution Check using Histogram
# plt.figure(figsize=(20,15))
# plt.hist(regressor.resid_pearson)
# plt.show()

# # Normal Distribution Check using Q-Q plot
# plt.figure(figsize=(20,15))
# st.probplot(regressor.resid_pearson, dist="norm", plot=pylab)
# plt.show()

# # Homoscedasticity : Error term being same across all values of independent variables
# # Fitted Values Vs Residuals
# plt.figure(figsize=(15,10))
# plt.scatter(preds,regressor.resid_pearson,c="r"),plt.axhline(y=0,color='blue');plt.xlabel("Fitted_Values");plt.ylabel("Residuals")

In [239]:
# Residual values
residuals = preds - modified_startup['Profit']

# RMSE value for the Dataset
rmse_value = np.sqrt(np.mean(residuals*residuals))

print("RMSE Value for the Dataset: ",rmse_value)
print("R-Square Value of the Model(Measure of Fit): ",regressor.rsquared)

RMSE Value for the Dataset:  7001.677262269533
R-Square Value of the Model(Measure of Fit):  0.9658237979414432
